In [0]:
pip install sdv pandas

In [0]:
# COMMAND ----------
# Install necessary libraries for Synthetic Data Generation
# Run this once in your cluster
%pip install sdv plotly matplotlib seaborn pandas scikit-learn

# COMMAND ----------
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# SDV (Synthetic Data Vault) Imports
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

In [0]:
# COMMAND ----------
# --- STEP 1: LOAD DATA ---

# NOTE: Update this path to where your file is stored in Databricks
# Common path structure: "/dbfs/FileStore/tables/Online_Sales_Data.csv"
# or if using Unity Catalog Volumes: "/Volumes/catalog/schema/volume/Online_Sales_Data.csv"
file_path = "/Workspace/Users/bchandrach@gmail.com/AdvDataAnalytics-Project-/Online Sales Data.csv"

try:
    # Try reading from the DBFS path
    real = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"File not found at {file_path}. Please check the path.")
    # Fallback for demonstration if file isn't found (creates a dummy frame or stops)
    raise

print("Real data loaded:", real.shape)
display(real.head())



In [0]:
# --- STEP 2: PREPROCESSING ---

# Convert Date to datetime for better handling
real['Date'] = pd.to_datetime(real['Date'])

# Separate columns that shouldn't be synthesized directly
# 1. Transaction ID is a key, we will regenerate it later.
# 2. Total Revenue is derived (Units * Price), we will recalculate it to ensure consistency.
training_data = real.drop(columns=['Transaction ID', 'Total Revenue'])

print("Training data prepared. Columns:", list(training_data.columns))

In [0]:
# --- STEP 3: SETUP SYNTHESIZER ---

# Detect metadata from the dataframe
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=training_data)

print("\nMetadata detected:")
print(metadata.to_dict())

# Initialize Gaussian Copula Synthesizer (Fast and effective for this size)
# You can switch to CTGANSynthesizer for more complex distributions if needed
synthesizer = GaussianCopulaSynthesizer(metadata)

In [0]:
# --- STEP 4: TRAIN AND SAMPLE ---

print("\nFitting synthesizer...")
synthesizer.fit(training_data)

print("\nGenerating synthetic data...")
# Generate the same number of rows as the real dataset
synth_data = synthesizer.sample(num_rows=1000)

In [0]:
# --- STEP 5: POST-PROCESSING ---

# 1. Recalculate Total Revenue to maintain mathematical consistency
synth_data['Total Revenue'] = synth_data['Units Sold'] * synth_data['Unit Price']

# 2. Regenerate Transaction ID (continuing from the original or starting fresh)
# Here we recreate IDs similar to the original format
start_id = real['Transaction ID'].min()
synth_data['Transaction ID'] = range(start_id, start_id + len(synth_data))

# Reorder columns to match original
synth_data = synth_data[real.columns]

# Display results
print("Synthetic data generated:", synth_data.shape)
display(synth_data.head())

# Save to DBFS (Optional)
output_path = "/Workspace/Users/bchandrach@gmail.com/AdvDataAnalytics-Project-/Synthetic_Online_Sales.csv"
synth_data.to_csv(output_path, index=False)
print(f"Synthetic data saved to: {output_path}")

In [0]:
# --- STEP 6: EVALUATION (PCA & METRICS) ---

def visualize_pca(real_df, synth_df, label):
    # Select numeric columns for PCA (exclude IDs)
    numeric_cols = ['Units Sold', 'Unit Price', 'Total Revenue']
    
    # Standardize data
    scaler = StandardScaler()
    X_real = scaler.fit_transform(real_df[numeric_cols])
    X_synth = scaler.transform(synth_df[numeric_cols])
    
    # Apply PCA
    pca = PCA(n_components=2, random_state=42)
    pcs_real = pca.fit_transform(X_real)
    pcs_synth = pca.transform(X_synth)
    
    # Plot
    plt.figure(figsize=(8,6))
    plt.scatter(pcs_real[:,0], pcs_real[:,1], alpha=0.5, label="Real", c='blue', s=20)
    plt.scatter(pcs_synth[:,0], pcs_synth[:,1], alpha=0.5, label="Synthetic", c='orange', s=20)
    plt.title(f"PCA — Real vs {label}")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.legend()
    plt.show()

# Run PCA Visualization
visualize_pca(real, synth_data, "Gaussian Copula Synthetic")

# Compare Statistics
print("\n--- Summary Statistics Comparison ---")
comp_stats = pd.concat([real.describe(), synth_data.describe()], axis=1, keys=['Real', 'Synthetic'])
print(comp_stats)

# Detectability (Silhouette Score)
def detectability_score(real_df, synth_df):
    numeric_cols = ['Units Sold', 'Unit Price', 'Total Revenue']
    
    combined = pd.concat([real_df[numeric_cols], synth_df[numeric_cols]])
    X = StandardScaler().fit_transform(combined)
    
    # If the data is distinguishable, KMeans will cluster them apart
    kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
    labels = kmeans.fit_predict(X)
    
    # A lower score usually indicates better blending (harder to distinguish)
    return silhouette_score(X, labels)

score = detectability_score(real, synth_data)
print(f"\nDetectability Silhouette Score: {score:.4f} (Lower is often better for privacy/blending)")

In [0]:

# --- Calculate Mean Absolute Correlation Difference ---

# 1. Select numeric columns for correlation analysis
# We exclude 'Transaction ID' as it's just a key, and 'Date' requires different handling
numeric_cols = ['Units Sold', 'Unit Price', 'Total Revenue']

# 2. Compute Correlation Matrices
real_corr = real[numeric_cols].corr()
synth_corr = synth_data[numeric_cols].corr()

# 3. Calculate Mean Absolute Difference
# This takes the absolute difference between every cell in the matrices and averages them
correlation_difference = (real_corr - synth_corr).abs().mean().mean()

print(f"Mean Absolute Correlation Difference: {correlation_difference:.4f}")

# --- (Optional) Visualize the Difference ---
# Plotting the difference matrix helps you see WHICH correlations are off
diff_matrix = (real_corr - synth_corr).abs()

plt.figure(figsize=(8, 6))
sns.heatmap(diff_matrix, annot=True, cmap='Reds', vmin=0, fmt=".3f")
plt.title("Absolute Correlation Difference (Real - Synthetic)")
plt.show()